# Explore SSRS files

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

In [ ]:
df = pd.read_json("../data/01_raw/sendaway/SSRS15-01_322012079/PGN129025_POSITION_LAT-LON_DSH_SA16/1635242819.txt.gz", compression='gzip', lines=True, orient="columns")

In [ ]:
df

In [ ]:
pd.to_datetime(df[['min_time_stamp','max_time_stamp']].mean(axis=1), unit='s')

In [ ]:
dir_path = "../data/01_raw/sendaway/SSRS15-01_322012079/PGN129025_POSITION_LAT-LON_DSH_SA13"
parts = []
for file in os.listdir(dir_path):
    file_path = os.path.join(dir_path, file)
    part = pd.read_json(file_path, compression='gzip', lines=True, orient="columns")
    parts.append(part)

In [ ]:
df = pd.concat(parts)
df.index = pd.to_datetime(df[['min_time_stamp','max_time_stamp']].mean(axis=1), unit='s')

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
type(df['last'].iloc[0])

In [ ]:
dir_path = "../data/01_raw/sendaway/SSRS15-01_322012079/"
parts = []
counter = 0
limit = 1000
for directory in os.listdir(dir_path):
    
   
    if 'SSRS' in directory:
        continue  # skipping these strange folders...
    
    sub_dir_path = os.path.join(dir_path, directory)
        
    for file in os.listdir(sub_dir_path):

    
    
        file_path = os.path.join(sub_dir_path, file)
        part = pd.read_json(file_path, compression='gzip', lines=True, orient="columns")
        
        for index,row in part.iterrows():
            
            content = row['decimated_signal']
            df_ = pd.DataFrame(index=content.keys())
            df_[str(row['signal'])] = [item['value'] for item in content.values()]
            df_['unit'] = row['unit']
                    
            parts.append(df_)
            
            counter+=1
        if counter > limit:
            break

    if counter > limit:
        break

In [ ]:
parts[0]

In [ ]:
directory

In [ ]:
def save_signal(row, name:str, data, values):
    
    content = row['decimated_signal']
    index = content.keys()
    
    s = pd.Series(data = values, index=index, name=name, dtype=float)
                
    s.index=s.index.astype(int)
    s.index = pd.to_datetime(s.index, unit='s')
    s.sort_index(inplace=True)
    
    s2 = s.resample('10S').interpolate()
    
    if not name in data:
        data[name]=s2
    else:
        data[name] = data[name].append(s2)

In [ ]:
dir_path = "../data/01_raw/sendaway/SSRS15-01_322012079/"
parts = []
counter = 0
limit = np.inf
indexes = []
names = []

excludes = [
    'ENGINEOILTEMP',
    'ENGINETEMP',
    'TOTALENGINEHOURS',
    'EngineOilPressure',
]

data = {}

for directory in os.listdir(dir_path):
    
    if np.any(np.array([exclude.lower() in directory.lower() for exclude in excludes])):
        continue
   
    if 'SSRS' in directory:
        continue  # skipping these strange folders...
    
    sub_dir_path = os.path.join(dir_path, directory)
        
    for file in os.listdir(sub_dir_path):

         
    
        file_path = os.path.join(sub_dir_path, file)
        part = pd.read_json(file_path, compression='gzip', lines=True, orient="columns")
        
        for index,row in part.iterrows():
                        
            name=str(row['signal'])
            content = row['decimated_signal']
            if 'LAT-LON' in name:
                
                name = 'latitude'
                values = [item['value'][0] for item in content.values()]
                save_signal(row=row, name=name, data=data, values=values)
                
                name = 'longitude'
                values = [item['value'][1] for item in content.values()]
                save_signal(row=row, name=name, data=data, values=values)
                
            else:
                values = [item['value'] for item in content.values()]
                
                save_signal(row=row, name=name, data=data, values=values)
            
            counter+=1
        if counter > limit:
            break

    if counter > limit:
        break

In [ ]:
data.keys()

In [ ]:
s1 = data['PGN127489_ENGINEPARAMETERSDYNAMIC_FuelRate_dsh_SA236_dsh_INST_1']
s2 = data['PGN127489_ENGINEPARAMETERSDYNAMIC_FuelRate_dsh_SA236_dsh_INST_0']


In [ ]:
s1

In [ ]:
s2

In [ ]:
for key,value in data.items():
    
    print(f"{pd.to_datetime(value.index.min(), unit='s')} : {key}")

In [ ]:
min_index = np.max([value.index.min() for value in data.values()])
min_index

In [ ]:
max_index = np.min([value.index.max() for value in data.values()])
max_index

In [ ]:
df

In [ ]:
df = pd.DataFrame()
for key,values in data.items():
    #mask = ((values.index >= min_index) & (values.index <= max_index) )
    #df[key] = values.loc[mask].values
    
    if len(df) == 0:
        df=values.to_frame()
    else:
        df = df.combine_first(values.to_frame())
    

In [ ]:
df

In [ ]:
df.describe()